In [1]:
import os

import json
import numpy as np


from langchain_community.vectorstores import FAISS
import faiss
import torch
# from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

# from abc import abstractmethod
# import concurrent.futures
# from concurrent.futures import ThreadPoolExecutor

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from dotenv import load_dotenv
from langchain_core.documents import Document
from typing_extensions import List, TypedDict

from sentence_transformers import SentenceTransformer


/Users/amarkanaka/miniconda3/envs/faiss_1.8.0_new_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
dataset = 'musique'
model_label = 'SBERT'
unit = 'hippo'

vector_path = f'data/{dataset}/{dataset}_{model_label}_{unit}_vectors_norm.npy'
index_path = f'data/{dataset}/{dataset}_{model_label}_{unit}_ip_norm.index'

#check if index has been built, otherwise build it using Sentence BERT
if(os.path.isfile(index_path)):
    if dataset == 'musique':
        faiss_index = faiss.read_index('data/musique/musique_facebook_contriever_proposition_ip_norm.index')
    else:
        faiss_index = faiss.read_index('data/2wikimultihopqa/2wikimultihopqa_facebook_contriever_proposition_ip_norm.index')
else:
    corpus_contents = []
    if dataset == 'musique':
        corpus = json.load(open('data/musique_corpus.json', 'r'))
    elif dataset == '2wikimultihopqa':
        corpus = json.load(open('data/2wikimultihopqa_corpus.json', 'r'))
    for item in corpus:
        corpus_contents.append(item['title'] + '\n' + item['text'])
    model = SentenceTransformer('bert-base-nli-mean-tokens')
    sentence_embeddings = model.encode(corpus_contents)
    sentence_embeddings.shape


